In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import models
from torch.autograd import Variable

from resnet_yolo import resnet50, resnet18
from dataset import yoloDataset
from eval_voc import *


import torch.nn.functional as F


# from visualize import Visualizer
import numpy as np
import os


In [2]:

file_root = '/home/stefan/data/VOCdevkit/VOC2007/JPEGImages/'
project_path = os.getcwd()
learning_rate = 0.001
num_epochs = 50
batch_size = 24
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Loading pretrained model

In [3]:
net = resnet18()
print('load model...')
net.load_state_dict(torch.load('best_multi.pth', map_location='cpu'))
net = net.to(device)
net.eval()

load model...


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

### Data loading

In [4]:
train_dataset = yoloDataset(root=file_root,list_file=os.path.join(project_path, 'voc2007.txt'),train=False,transform = [transforms.ToTensor()] )


data init


In [5]:
test_eval()

---class cat ap 0.8333333333333333---
---class dog ap 1.0---
---map 0.9166666666666666---


### Evaluate VOC map

In [6]:
from predict import *
from collections import defaultdict
from tqdm import tqdm

target =  defaultdict(list)
preds = defaultdict(list)
image_list = [] #image path list

f = open(os.path.join(project_path, 'voc2007test.txt'))
lines = f.readlines()
file_list = []
for line in lines:
    splited = line.strip().split()
    file_list.append(splited)
f.close()
print('---prepare target---')
for index,image_file in enumerate(file_list):
    image_id = image_file[0]

    image_list.append(image_id)
    num_obj = (len(image_file) - 1) // 5
    for i in range(num_obj):
        x1 = int(image_file[1+5*i])
        y1 = int(image_file[2+5*i])
        x2 = int(image_file[3+5*i])
        y2 = int(image_file[4+5*i])
        c = int(image_file[5+5*i])
        class_name = VOC_CLASSES[c]
        target[(image_id,class_name)].append([x1,y1,x2,y2])


---prepare target---


In [7]:
#
#start test
#
print('---start test---')

count = 0
for image_path in tqdm(image_list):
    result = predict_img(net, image_path, device, root_path=file_root) #result[[left_up,right_bottom,class_name,image_path],]
    for (x1,y1),(x2,y2),class_name,image_id,prob in result: #image_id is actually image_path
        preds[class_name].append([image_id,prob,x1,y1,x2,y2])
    # print(image_path)
    # image = cv2.imread('/home/xzh/data/VOCdevkit/VOC2012/allimgs/'+image_path)
    # for left_up,right_bottom,class_name,_,prob in result:
    #     color = Color[VOC_CLASSES.index(class_name)]
    #     cv2.rectangle(image,left_up,right_bottom,color,2)
    #     label = class_name+str(round(prob,2))
    #     text_size, baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)
    #     p1 = (left_up[0], left_up[1]- text_size[1])
    #     cv2.rectangle(image, (p1[0] - 2//2, p1[1] - 2 - baseline), (p1[0] + text_size[0], p1[1] + text_size[1]), color, -1)
    #     cv2.putText(image, label, (p1[0], p1[1] + baseline), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,255,255), 1, 8)

    # cv2.imwrite('testimg/'+image_path,image)
    count += 1
    if count == 200:
        break

print('---start evaluate---')
voc_eval(preds,target,VOC_CLASSES=VOC_CLASSES)

  0%|          | 0/4951 [00:00<?, ?it/s]

---start test---


/home/stefan/programs/anaconda3/envs/ssd/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
  4%|▍         | 199/4951 [01:57<45:24,  1.74it/s]  

---start evaluate---
---class aeroplane ap 0.004678362573099415---
---class bicycle ap 0.0057227638830012716---
---class bird ap 0.0019344902712042672---
---class boat ap 0.0---
---class bottle ap 0.0021321961620469083---
---class bus ap 0.006259780907668232---
---class car ap 0.01717432060543516---
---class cat ap 0.007649822244794312---
---class chair ap 0.001300705467372134---
---class cow ap 0.0---
---class diningtable ap 0.0---
---class dog ap 0.008439640341480833---
---class horse ap 0.003694581280788177---
---class motorbike ap 0.006153846153846154---
---class person ap 0.0032053811788276257---
---class pottedplant ap 0.0---
---class sheep ap 0.0---
---class sofa ap 0.0---
---class train ap 0.006382978723404256---
---class tvmonitor ap 0.0016233766233766235---
---map 0.003817612320817268---
